# Chap3 Deep Learning with Pytorch

## Tensors

In [1]:
import torch
import numpy as np

In [2]:
a = torch.FloatTensor(3, 2)
a

tensor([[0.0000e+00, 0.0000e+00],
        [5.2002e+22, 1.7664e-04],
        [2.6563e+20, 3.2722e+21]])

In [3]:
a.zero_()

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])